In [1]:
import pandas as pd
import numpy as np
import glob
import os
from bokeh.plotting import figure, output_notebook, show, gridplot
from bokeh.palettes import Category10
from bokeh.charts import Bar, Area

In [43]:
epochs = 250000
directory="/notebooks/admin/saves/stats/gamma_test-size-5-epochs-250000-date-2017-01-23/"
first_player = False

In [3]:
l = [pd.read_hdf(filename) for filename in glob.glob(directory + "/*.h5")]
df = pd.concat(l, axis=0)

In [46]:
#Fix for first player
df['first_player'] = df['first_player'].astype(bool)

In [47]:
df.head()

,early_reward,epoch,epsilon,first_player,gamma,loss,move_count,random_ai_move,random_move,reward,size,winner
0,False,0.0,NaN,False,0.0,NaN,1.0,True,True,NaN,5,-1.0
1,False,0.0,1.0,False,0.0,0.000349,2.0,False,True,0.0,5,-1.0
2,False,0.0,NaN,False,0.0,NaN,3.0,True,True,NaN,5,-1.0
3,False,0.0,1.0,False,0.0,0.003178,4.0,False,True,0.0,5,-1.0
4,False,0.0,NaN,False,0.0,NaN,5.0,True,True,NaN,5,-1.0


In [79]:
p = figure(
    title="loss versus gamma",
    x_axis_label='epoch',
    y_axis_label='loss',
    plot_width=800,
    webgl=True)

for gamma in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    mask = (df.gamma == gamma) & (df.random_ai_move == False) & \
           (df.random_move == False) & (df.epoch % 100 == 0)

    p.circle(
        df[mask].epoch,
        df[mask].loss,
        legend="gamma=%f" % gamma,
        alpha=0.5,
        line_color=None,
        color=Category10[10][int(gamma * 10)] if gamma != 1 else 'black')

output_notebook()
show(p)

In [80]:
figures = []
for gamma in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    p = figure(
        title="loss versus gamma",
        x_axis_label='epoch',
        y_axis_label='loss',
        plot_width=400,
        plot_height=400,
        webgl=True)

    mask = (df.gamma == gamma) & (df.random_ai_move == False) & (
        df.random_move == False) & (df.epoch % 100 == 0)

    p.circle(
        df[mask].epoch,
        df[mask].loss,
        legend="gamma=%f" % gamma,
        alpha=0.5,
        line_color=None,
        color=Category10[10][int(gamma * 10)] if gamma != 1 else 'black')

    if gamma != 0:
        p.x_range = figures[0].x_range
        p.y_range = figures[0].y_range

    figures.append(p)

p = gridplot([[figures[i], figures[i + 1]]
              for i in range(0, 10, 2)] + [[figures[-1]]])
output_notebook()
show(p)

In [51]:
figures = []
for gamma in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    t = df[(df.gamma == gamma) & (df.winner != -1)][['epoch', 'winner']]

    t = t.set_index(['epoch'])
    t['won'] = t['winner'] == (0 if first_player else 1)
    t['lost'] = t['winner'] == (1 if first_player else 0)
    t['error'] = t['winner'] == 2
    t.drop('winner', 1, inplace=True)
    k = 1000
    t = t.groupby(pd.cut(t.index, np.arange(0, epochs + k, k))).sum()
    t.index = [i * k for i in range(len(t))]

    p = Area(
        t,
        title="gamma=" + str(gamma),
        xlabel='epoch',
        ylabel='winner count',
        stack=True,
        plot_width=400,
        plot_height=400,
        webgl=True)

    if gamma != 0:
        p.x_range = figures[0].x_range
        p.y_range = figures[0].y_range

    figures.append(p)

p = gridplot([[figures[i], figures[i + 1]]
              for i in range(0, 10, 2)] + [[figures[-1]]])
output_notebook()
show(p)

Loading BokehJS ...